## Installation and imports




In [ ]:
# install DLOmix with wandb extras
#!python -m pip install dlomix[wandb] >= 0.0.5
#!python -m pip install git+https://github.com/wilhelm-lab/dlomix

In [ ]:
# import necessary packages
import tensorflow as tf

import dlomix
from dlomix import (
    constants, data, eval,
    layers, models, pipelines, reports, utils
)
from dlomix.data import RetentionTimeDataset
from dlomix.models import RetentionTimePredictor
from dlomix.eval import TimeDeltaMetric

# Initialize Weights & Biases


In [ ]:
import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger


# Create config
config = {
  "seq_length" : 30,
  "batch_size" : 64,
  "val_ratio" : 0.2,
  "lr" : 0.0001,
  "optimizer" : "Adam",
  "loss" : "mse"
}

# Initialize WANDB
PROJECT = 'Demo_ComparingModelsReport'
RUN = "run_1"
wandb.init(project = PROJECT, name = RUN, config = config)

# Load data

In [ ]:
# load small train dataset
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'

# create dataset
rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = config["batch_size"],
                              val_ratio = config["val_ratio"],
                              test = False)

# Train and save several models

In [ ]:
# create retention time predictor
model = RetentionTimePredictor(encoder="lstm", seq_length = config["seq_length"])

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate = config["lr"])

# compile the model  with the optimizer and the metrics we want to use,
# we can add our custom timedelta metric
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

# train the model
history = model.fit(rtdata.train_data,
                    validation_data = rtdata.val_data,
                    epochs=2,
                    callbacks=[WandbMetricsLogger(log_freq = "batch")])

# finish wandb run
wandb.finish()

# save the model weights
save_path = "./output/rtmodel_1"
model.save_weights(save_path)

In [ ]:
RUN = "run_2"
wandb.init(project = PROJECT, name = RUN, config = config)

In [ ]:
# create retention time predictor
model = RetentionTimePredictor(seq_length = config["seq_length"])

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate = config["lr"])

# compile the model  with the optimizer and the metrics we want to use,
# we can add our custom timedelta metric
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

# train the model
history = model.fit(rtdata.train_data,
                    validation_data = rtdata.val_data,
                    epochs=2,
                    callbacks=[WandbMetricsLogger(log_freq = "batch")])

# finish wandb run
wandb.finish()

# save the model weights
save_path = "./output/rtmodel_2"
model.save_weights(save_path)

# Load models and create testset

In [ ]:
# create predictors from saved weights
load_path = "./output/rtmodel_1"
pre_trained_model_1 = RetentionTimePredictor(encoder = "lstm", seq_length=30)
pre_trained_model_1.load_weights(load_path)

load_path = "./output/rtmodel_2"
pre_trained_model_2 = RetentionTimePredictor(seq_length=30)
pre_trained_model_2.load_weights(load_path)

models = {"lstm_model": pre_trained_model_1, "conv_model": pre_trained_model_2}

# create testset
TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix/main/example_dataset/proteomTools_test.csv'
test_set = RetentionTimeDataset(data_source = TEST_DATAPATH,
                                seq_length = config["seq_length"],
                                batch_size = 32,
                                test = True,
                                sequence_col = "sequence",
                                target_col = "irt")

# Initialize report

In [ ]:
# Create a report
from dlomix.reports import RetentionTimeReportModelComparisonWandb

report = RetentionTimeReportModelComparisonWandb(models = models,
                test_dataset = test_set,
                project = PROJECT,
                title = "Comparison of models",
                description = "A quick comparison of two models")

# Log data to Weights & Biases

In [ ]:
report.log_data()

# Compare models and create report

In [ ]:
report.compare_models()

In [ ]:
report.create_report(add_data_section=True,
                     add_residuals_section = True,
                     add_r2_section = True,
                     add_density_section= True)